# Traitement des données

Dans cette partie, l'objectif est d'importer nos données, les convertir à un format convenable pour la production de statistiques descriptives, et exporter le résultat pour que ces bases puissent être utilisées par les autres programmes

Les bases en question sont :
- la base listant tous les incidents de violence par armes à feu aux USA entre 2013 et 2018
- la base listant les caractéristiques économiques générales des comtés et de ses habitants.
- la base gun_legislation proposant une synthèse des différences de législation entre Etats
- la base bg_checks recensant tous les *background checks* du FBI pour l'année 2018 (documentation fournie plus bas)

In [1]:
#Pour le traitement classique
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon
from tqdm import tqdm

#Pour l'interaction avec l'API 
import requests
from statistics import mean
import time
from collections import defaultdict

## Traitement de la base d'incidents armes à feu

In [68]:
#Base incidents armes à feu
url="https://drive.google.com/file/d/1GGOLMc_Ow9yZC9sICegPegDggQuHOD3t/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
gun_violence_db = pd.read_csv(url)
gun_violence_db.sample(5)

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
25819,189701,2014-07-12,Georgia,Fort Benning,NaN,1,0,http://www.gunviolencearchive.org/incident/189701,http://www.wtvm.com/story/26511605/a-veterans-...,False,...,0::43,0::Adult 18+,0::Male,0::Donovan Hernandez,NaN,0::Killed,0::Victim,http://www.wtvm.com/story/26511605/a-veterans-...,138.0,15.0
108395,487427,2016-01-18,Arkansas,Lepanto,Staton Lane,0,0,http://www.gunviolencearchive.org/incident/487427,http://www.kait8.com/story/30991705/police-res...,False,...,NaN,0::Adult 18+,NaN,NaN,NaN,0::Unharmed,0::Subject-Suspect,http://www.kait8.com/story/30991705/police-res...,54.0,22.0
150892,675320,2016-10-13,Florida,Jacksonville,2600 Ribault Scenic Drive,0,1,http://www.gunviolencearchive.org/incident/675320,http://callsforservice.jaxsheriff.org/,False,...,NaN,NaN,NaN,NaN,NaN,0::Injured,0::Victim,http://callsforservice.jaxsheriff.org/,14.0,9.0
141896,635476,2016-08-20,Iowa,Sioux City,2025 Wall Street,0,2,http://www.gunviolencearchive.org/incident/635476,http://siouxcityjournal.com/news/local/sioux-c...,False,...,2::19,2::Adult 18+,0::Male||1::Male||2::Male,0::Daniel Perry||1::Michael Davis||2::Lasharbe...,NaN,"0::Injured||1::Injured||2::Unharmed, Arrested",0::Victim||1::Victim||2::Subject-Suspect,http://siouxcityjournal.com/news/local/briefs/...,NaN,NaN
231506,1043726,2018-02-05,Ohio,Miamisburg,125 Byers Rd,0,1,http://www.gunviolencearchive.org/incident/104...,http://www.whio.com/news/suspected-teen-robber...,False,...,0::17,0::Teen 12-17,0::Male,NaN,NaN,0::Injured,0::Subject-Suspect,http://www.whio.com/news/local/reports-one-sho...,42.0,6.0


Conformément à la documentation de la base, certaines colonnes sont codées de façon à pouvoir les reconvertir en dictionnaire :

In [69]:
def convert_to_dict(value):
    if pd.isna(value):
        return value

    pairs = value.split('||')
    result_dict = {}
    for pair in pairs:
        #Some are corrupted : 1: instead of ::
        if '::' in pair:
            key, val = pair.split('::', 1)
            result_dict[int(key)] = val
        else:
            key, val = pair.split(':', 1)
            result_dict[int(key)] = val
    return result_dict

list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
gun_violence_db[list_of_dict_columns] = gun_violence_db[list_of_dict_columns].applymap(convert_to_dict)
gun_violence_db.head()



/tmp/ipykernel_407416/4075116675.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gun_violence_db[list_of_dict_columns] = gun_violence_db[list_of_dict_columns].applymap(convert_to_dict)


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: 'Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: 'Male'},{0: 'Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{3: 'Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [70]:
gun_violence_db.to_csv("data/gun_violence_db.csv", index=False)

## Traitements des informations par comté

Ici, nous récupérons plusieurs informations nous permettant d'enrichir notre analyse des incidents par armes à feu. Nous utilisons l'API Fred, nous permettant de récupérer les valeurs de plusieurs séries temporelles de variables économiques par comté.
Puis, nous utilisons encore Fred pour récupérer la géographie des comtés, nécessaire pour effectuer de la cartographie.
La documentation précisant le mode d'interaction avec l'API de StLouisFed se trouve à la page https://fred.stlouisfed.org/docs/api/fred/#API.

### Préparation de l'extraction de données

Chaque objet dans l'API peut être associé avec son ID. Nous récupérons ici la liste des comtés et les ID associés.

In [5]:
#Each request is categorized with an url and an id
#The gist here is to recover the proper id to retrieve data
api_key = "180de2e6a1d1e953d270ebf38341cd44"
param = {"api_key" : api_key, "file_type" : "json", "category_id" : "27281"}
url = "https://api.stlouisfed.org/fred/category/children?"

In [6]:
def request_db(index):
    #this function requests to the API the database associated with the category id index
    param["category_id"] = index #on ajuste les paramètres de la request pour demander la bonne catégorie
    response = requests.get(url, params = param)
    data = response.json()
    return data

In [7]:
def check_response(url, param):
    while True:
        response = requests.get(url, params=param).json()

        if not response.get('error_message') or response['error_message'] != 'Too Many Requests.  Exceeded Rate Limit':
            return response
        time.sleep(5)

In [87]:
#Some names are ambiguous between dframes
def simplify_name(name):
    if name.endswith("County"):
        return name.rsplit("County", 1)[0].strip()
    if name.endswith("Parish"):
        return name.rsplit("Parish", 1)[0].strip()
    if name.endswith("Borough"):
        return name.rsplit("Borough", 1)[0].strip()
    if name.endswith("Census Area"):
        return name.rsplit("Census Area", 1)[0].strip()
    if name.endswith("City"):
        return name.rsplit("City", 1)[0].strip() 
    if name.endswith("Borough/city"):
        return name.rsplit("Borough/city", 1)[0].strip()      
    if name.endswith("Municipality"):
        return name.rsplit("Municipality", 1)[0].strip()     
    if name.endswith("Borough/municipality"):
        return name.rsplit("Borough/municipality", 1)[0].strip()      
    if name.endswith("County/city"):
        return name.rsplit("County/city", 1)[0].strip()
    if name.endswith("Counties"):
        return name.rsplit("Counties", 1)[0].strip()

        
    return name

In [88]:
us_data = request_db(27281)['categories']
#We create our dframe by creating a list of dicts, each element is a new row
database = list()
for state in tqdm(us_data):
    id_state = state['id']
    state_name = state['name']
    
    #Request to recover id in order to extract counties
    state_info = request_db(id_state)["categories"]
    if state_info != []: #One exception : which one ?
        id_list_of_state_counties = state_info[0]['id']
        list_of_state_counties = request_db(id_list_of_state_counties)["categories"]
        for county in list_of_state_counties:
            dict_county = dict()
            id_county = county['id']
            
            parts = county['name'].split(', ')
            county_name, state_code = parts[0], parts[-1]
            
            dict_county = {
                'Nom': simplify_name(county_name),
                'Etat': state_name,
                'Code_Etat': state_code,
                'id_Etat': id_state,
                'id_county': id_county
            }
            
            if '+' in county_name:
                county_name, county_bis = county_name.split(' + ')
                dict_county['Nom'] = simplify_name(county_bis)
                database.append(dict_county.copy())  # Append a copy to avoid modifying the original
                dict_county['Nom'] = simplify_name(county_name)
            database.append(dict_county)

counties_db = pd.DataFrame(database)

100%|██████████| 53/53 [00:21<00:00,  2.45it/s]


In [92]:
#Gestion des exceptions pour le merging(obligé de faire du cas par cas car merging sur le code_Etat + nom)

counties_db.loc[(counties_db['Code_Etat'] == 'Aleutian Islands Census Area'), 'Code_Etat'] = 'AK'
counties_db.loc[(counties_db['Code_Etat'] == 'District of Columbia'), 'Code_Etat'] = 'DC'
counties_db.loc[(counties_db['Nom'] == 'De Soto'), 'Nom'] = 'DeSoto'
counties_db.loc[(counties_db['Nom'] == 'DeSoto') & (counties_db['Code_Etat'] == 'LA'), 'Nom'] = 'De Soto'
counties_db.loc[(counties_db['Nom'] == 'De Kalb'), 'Nom'] = 'DeKalb'
counties_db.loc[(counties_db['Nom'] == 'Du Page'), 'Nom'] = 'DuPage'
counties_db.loc[(counties_db['Nom'] == 'La Salle'), 'Nom'] = 'LaSalle'
counties_db.loc[(counties_db['Nom'] == 'La Porte'), 'Nom'] = 'LaPorte'
counties_db.loc[(counties_db['Nom'] == 'Lagrange'), 'Nom'] = 'LaGrange'
counties_db.loc[(counties_db['Nom'] == 'LaFourche'), 'Nom'] = 'Lafourche'
counties_db.loc[(counties_db['Nom'] == 'Carson') & (counties_db['Code_Etat'] == 'NV'), 'Nom'] = 'Carson City'
counties_db.loc[(counties_db['Nom'] == 'Lac Qui Parle'), 'Nom'] = 'Lac qui Parle'
counties_db.loc[(counties_db['Nom'] == 'Dona Ana'), 'Nom'] = 'Doña Ana'
counties_db.loc[(counties_db['Nom'] == 'La Moure'), 'Nom'] = 'LaMoure'
counties_db.loc[(counties_db['Nom'] == 'De Witt'), 'Nom'] = 'DeWitt'
counties_db.loc[(counties_db['Nom'] == 'DeWitt') & (counties_db['Code_Etat'] == 'IL'), 'Nom'] = 'De Witt'
counties_db.loc[(counties_db['Nom'] == 'LaSalle') & (counties_db['Code_Etat'] == 'TX'), 'Nom'] = 'La Salle'
counties_db.loc[(counties_db['Code_Etat'] == 'WI (includes Menominee)'), 'Code_Etat'] = 'WI'
counties_db.loc[(counties_db['Nom'] == 'Fond Du Lac'), 'Nom'] = 'Fond du Lac'

On a maintenant un premier dframe recensant tous les comtés des USA ainsi que les ID permettant de les retrouver dans l'API. On peut désormais extraire pour chaque comté les informations socio-démographiques nous permettant de produire nos statistiques descriptives et notre modèle.

Note : id_county fait office de clé primaire dans cette base (au sein de l'API FRED)

In [77]:
counties_db.to_csv("data/counties_db.csv", index=False)

### Récupération des données géographiques

FRED n'inclut pas directement de données géographiques associées aux comtés par leurs ID. Il faut donc ici récupérer ces données géographiques, et effectuer une jointure sur l'Etat et le nom du comté (donc de nombreuses exceptions).

In [102]:
geo_counties = gpd.read_file('data/cb_2018_us_county_500k/')

In [103]:
state_fips_to_name = {
    '01': 'Alabama',
    '02': 'Alaska',
    '04': 'Arizona',
    '05': 'Arkansas',
    '06': 'California',
    '08': 'Colorado',
    '09': 'Connecticut',
    '10': 'Delaware',
    '11': 'District of Columbia',
    '12': 'Florida',
    '13': 'Georgia',
    '15': 'Hawaii',
    '16': 'Idaho',
    '17': 'Illinois',
    '18': 'Indiana',
    '19': 'Iowa',
    '20': 'Kansas',
    '21': 'Kentucky',
    '22': 'Louisiana',
    '23': 'Maine',
    '24': 'Maryland',
    '25': 'Massachusetts',
    '26': 'Michigan',
    '27': 'Minnesota',
    '28': 'Mississippi',
    '29': 'Missouri',
    '30': 'Montana',
    '31': 'Nebraska',
    '32': 'Nevada',
    '33': 'New Hampshire',
    '34': 'New Jersey',
    '35': 'New Mexico',
    '36': 'New York',
    '37': 'North Carolina',
    '38': 'North Dakota',
    '39': 'Ohio',
    '40': 'Oklahoma',
    '41': 'Oregon',
    '42': 'Pennsylvania',
    '44': 'Rhode Island',
    '45': 'South Carolina',
    '46': 'South Dakota',
    '47': 'Tennessee',
    '48': 'Texas',
    '49': 'Utah',
    '50': 'Vermont',
    '51': 'Virginia',
    '53': 'Washington',
    '54': 'West Virginia',
    '55': 'Wisconsin',
    '56': 'Wyoming',
}

In [104]:
geo_counties = geo_counties[['STATEFP', 'COUNTYFP', 'NAME', 'geometry']]
geo_counties['fips'] = geo_counties['STATEFP'] + geo_counties['COUNTYFP']
geo_counties['state'] = geo_counties['STATEFP'].map(state_fips_to_name)
geo_counties = geo_counties.drop(columns=['STATEFP', 'COUNTYFP'])

In [105]:
#Now we can merge on 'Nom' and 'Etat'
counties_db = pd.merge(counties_db, geo_counties, left_on=['Nom', 'Etat'], right_on=['NAME', 'state'], how='left')

Comtés où la jointure géographique n'est pas possible :

In [106]:
counties_db.loc[merged_db['NAME'].isna()].head(40)

,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,state
68,Aleutian Islands,Alaska,AK,27283,33743,NaN,None,NaN,NaN
90,Prince of Wales-Outer Ketchikan,Alaska,AK,27283,27421,NaN,None,NaN,NaN
92,Skagway-Hoonah-Angoon,Alaska,AK,27283,27423,NaN,None,NaN,NaN
96,Wade Hampton,Alaska,AK,27283,27426,NaN,None,NaN,NaN
97,Wrangell Borough/,Alaska,AK,27283,33518,NaN,None,NaN,NaN
98,Wrangell-Petersburg,Alaska,AK,27283,27427,NaN,None,NaN,NaN
99,Yakutat City and,Alaska,Yakutat City and Borough,27283,32212,NaN,None,NaN,NaN
2429,Shannon,South Dakota,SD,27324,29791,NaN,None,NaN,NaN
2860,Clifton Forge,Virginia,VA,27330,30228,NaN,None,NaN,NaN
2957,South Boston,Virginia,VA,27330,32143,NaN,None,NaN,NaN


### Récupération des séries FRED

Ici, le but est de récupérer des données économiques clé pour chaque comté : population, taux de chômage, salaire médian, nombre de bénéficiaires des aides alimentaires, taux de scolarisation.
En raison de la limite de requêtes par minute, nous décomposons cette extraction pour minimiser les erreurs associées.
Dans un premier temps, nous récupérons la liste des séries associées à chaque comté ainsi que l'ID de ces séries.
Une fois l'ID récupéré, nous extrayons toutes les données nécessaires.

In [16]:
#Attempt to optimize the execution time + retrieve multiple series all at once
#Step 1 : Retrieve the ID of all series
dict_series = {}
for id_county in tqdm(counties_db['id_county']):
    url = "https://api.stlouisfed.org/fred/category/series?"
    param = {"api_key" : api_key, "file_type" : "json", "category_id" : id_county}
    response = check_response(url, param)

    dict_series[id_county] = response['seriess']

100%|██████████| 3195/3195 [26:23<00:00,  2.02it/s]  


In [17]:
#Step 2 : Retrieve the ID of the specific series we want to study
series_to_retrieve = {}
for id_county in tqdm(counties_db['id_county']):
    unemp_rate_id = med_h_income_id = resident_pop_id = snap_beneficiaries_id = bachelors_deg_id = 'not found'
    for serie in dict_series[id_county]:
        if ("Unemployment Rate" in serie["title"]) and ("Monthly" in serie["frequency"]):
            unemp_rate_id = serie['id']
        if ("Estimate of Median Household Income" in serie["title"]) and ("Interval" not in serie["title"]):
            med_h_income_id = serie['id']
        if ("Resident Population" in serie["title"]):
            resident_pop_id = serie['id']
        if ("SNAP Benefits Recipients" in serie["title"]):
            snap_beneficiaries_id = serie['id']
        if ("Bachelor's Degree or Higher" in serie["title"]):  
            bachelors_deg_id = serie['id']
    series_to_retrieve[id_county] = (unemp_rate_id, med_h_income_id, resident_pop_id, snap_beneficiaries_id, bachelors_deg_id)
#eg : series_to_retrieve = {id_county_1 : (series_1_id, series_2_id, ...)}

100%|██████████| 3195/3195 [00:00<00:00, 4279.11it/s]


In [31]:
#Step 3 : Determine the value associated with the series
list_of_values = defaultdict(dict)
list_of_variables = ['unemp_rate', 'med_h_income', 'resident_pop', 'snap_beneficiaries', 'bachelors_deg']

for id_county in tqdm(counties_db['id_county']):
    for index, id_series in enumerate(series_to_retrieve[id_county]):
        if id_series == 'not found':
            continue

        # Request data
        url = "https://api.stlouisfed.org/fred/series/observations?"
        param = {
            "api_key": api_key,
            "file_type": "json",
            "series_id": id_series,
            "observation_start": "2013-01-01",
            "observation_end": "2018-01-01",
        }
        observations = check_response(url, param).get("observations", [])

        # Extract values and years
        values = [float(obs["value"]) for obs in observations]
        years = [pd.to_datetime(obs["date"]).year for obs in observations]

        # Save values for each year
        for year, value in zip(years, values):
            list_of_values[id_county][f"{list_of_variables[index]}_year_{year}"] = value

# Convert defaultdict to regular dict for better compatibility
list_of_values = dict(list_of_values)

100%|██████████| 3195/3195 [2:11:06<00:00,  2.46s/it]  


In [97]:
#Step 4 : Convert data in a dataframe, then merge it to obtain our final db
cols = ['unemp_rate_year_2013', 'unemp_rate_year_2014', 'unemp_rate_year_2015', 'unemp_rate_year_2016', 'unemp_rate_year_2017', 'unemp_rate_year_2018', 'med_h_income_year_2013', 'med_h_income_year_2014', 'med_h_income_year_2015', 'med_h_income_year_2016', 'med_h_income_year_2017', 'med_h_income_year_2018', 'resident_pop_year_2013', 'resident_pop_year_2014', 'resident_pop_year_2015', 'resident_pop_year_2016', 'resident_pop_year_2017', 'resident_pop_year_2018', 'snap_beneficiaries_year_2013', 'snap_beneficiaries_year_2014', 'snap_beneficiaries_year_2015', 'snap_beneficiaries_year_2016', 'snap_beneficiaries_year_2017', 'snap_beneficiaries_year_2018', 'bachelors_deg_year_2013', 'bachelors_deg_year_2014', 'bachelors_deg_year_2015', 'bachelors_deg_year_2016', 'bachelors_deg_year_2017', 'bachelors_deg_year_2018']
fred_db = pd.DataFrame.from_dict(list_of_values, orient='index', columns=cols)
fred_db.reset_index(inplace=True)
fred_db.rename(columns={'index': 'id_county'}, inplace=True)
fred_db.head(5)

counties_db = pd.merge(left= counties_db, right=fred_db, how='left', left_on='id_county', right_on='id_county')

In [115]:
fred_db = pd.read_csv('data/final_db.csv')
fred_db = fred_db.loc[:, ['id_county'] + list(fred_db.loc[:, 'unemp_rate_year_2013':'bachelors_deg_year_2018'])]
counties_db = pd.merge(left= counties_db, right=fred_db, how='left', left_on='id_county', right_on='id_county')

In [135]:
counties_db

,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,unemp_rate_year_2013,unemp_rate_year_2014,...,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018
0,Autauga,Alabama,AL,27282,27336,Autauga,"POLYGON ((-86.92120 32.65754, -86.92035 32.658...",01001,5.9,4.8,...,8015.0,7844.0,7696.0,7477.0,20.9,21.9,23.2,24.6,25.0,27.7
1,Baldwin,Alabama,AL,27282,27337,Baldwin,"POLYGON ((-88.02858 30.22676, -88.02399 30.230...",01003,6.5,5.6,...,22920.0,21406.0,20241.0,19499.0,27.7,28.6,29.0,29.5,30.7,31.3
2,Barbour,Alabama,AL,27282,27338,Barbour,"POLYGON ((-85.74803 31.61918, -85.74544 31.618...",01005,9.6,8.8,...,6730.0,6675.0,6124.0,5898.0,13.4,13.6,12.5,12.9,12.0,12.2
3,Bibb,Alabama,AL,27282,27339,Bibb,"POLYGON ((-87.42194 33.00338, -87.31854 33.006...",01007,7.3,6.1,...,3794.0,3740.0,3504.0,3263.0,12.1,10.2,10.6,12.0,13.2,11.5
4,Blount,Alabama,AL,27282,27340,Blount,"POLYGON ((-86.96336 33.85822, -86.95967 33.857...",01009,6.1,5.1,...,7889.0,7037.0,6402.0,5760.0,12.1,12.3,12.9,13.0,13.1,12.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,Sweetwater,Wyoming,WY,27334,30524,Sweetwater,"POLYGON ((-110.05438 42.01103, -110.05436 42.0...",56037,4.4,3.5,...,2117.0,2095.0,1733.0,1496.0,18.1,18.1,19.6,21.4,22.2,22.0
3246,Teton,Wyoming,WY,27334,30525,Teton,"POLYGON ((-111.05361 44.66627, -110.75076 44.6...",56039,6.7,4.5,...,148.0,103.0,80.0,63.0,48.7,51.9,53.9,53.8,54.1,57.4
3247,Uinta,Wyoming,WY,27334,30526,Uinta,"POLYGON ((-111.04662 41.15604, -111.04659 41.2...",56041,5.3,4.0,...,1566.0,1500.0,1277.0,1178.0,18.9,18.7,19.3,18.2,17.4,15.4
3248,Washakie,Wyoming,WY,27334,30527,Washakie,"POLYGON ((-108.55056 44.16845, -108.50652 44.1...",56043,4.6,3.9,...,504.0,481.0,403.0,369.0,23.6,21.2,21.1,21.1,21.0,21.5


## Traitement de la base sur la législation par Etat

La base ici étudiée nous permet de synthétiser les différences de législation encadrant la vente, la possession et l'utilisation d'armes à feu sur le teritoire américain. Pour pouvoir comparer ces différences, nous essayons ici de construire des variables synthétiques traduisant l'intensité des restrictions dans plusieurs catégories : ....

La documentation de la base peut se trouver à l'addresse https://view.officeapps.live.com/op/view.aspx?src=https%3A%2F%2Fmail.statefirearmlaws.org%2Fsites%2Fdefault%2Ffiles%2F2020-07%2Fcodebook_0.xlsx&wdOrigin=BROWSELINK.

In [136]:
legislation_db = pd.read_excel('data/gun_legislation.xlsx')
legislation_db = legislation_db.loc[legislation_db['year'] == 2018].sort_values(by='state')
legislation_db.head(10)

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
27,Alabama,2018,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,10
57,Alaska,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
87,Arizona,2018,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,8
117,Arkansas,2018,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
147,California,2018,1,1,0,1,0,0,1,1,...,1,1,1,1,1,1,1,1,1,109
177,Colorado,2018,1,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,30
207,Connecticut,2018,1,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,0,1,88
237,Delaware,2018,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,41
267,Florida,2018,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,30
297,Georgia,2018,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


Ici, pour synthétiser l'information, nous pouvons construire un score d'intensité de la régulation pour chaque indicatrice, et calculer le score total pour cahque état (en groupant les indicatrices par type de législation).
Les scores sont ici fixés de manière arbitraire, se rangeant de 1(législation modérée) à 5(législation très restrictive). Nous calculons ensuite le maximum de score pour chaque catégorie de législation :
- réglementation des vendeurs
- réglementation des acheteurs
- restriction des armes à haut risque
- *background checks*
- réglementation de la possession d'armes à feu
- réglementation des armes d'assaut
- protetion des enfants
- réglementation du traffic d'armes
- restrictions après des faits de violence domestique


In [137]:
# Adjusted scores for each category
category_scores = {
    'dealer_regulations': {
        'dealer': 5,
        'dealerh': 4,
        'recordsdealer': 3,
        'recordsdealerh': 3,
        'recordsall': 2,
        'recordsallh': 2,
        'reportdealer': 5,
        'reportdealerh': 4,
        'reportall': 3,
        'reportallh': 3,
        'purge': 2,
        'residential': 1,
        'theft': 3,
        'security': 4,
        'inspection': 3,
        'liability': 4,
        'junkgun': 5,
    },
    'buyers_regulations': {
        'waiting': 4,
        'waitingh': 3,
        'permit': 5,
        'permith': 4,
        'permitlaw': 3,
        'fingerprint': 4,
        'training': 3,
        'registration': 5,
        'registrationh': 4,
        'defactoreg': 3,
        'defactoregh': 3,
        'age21handgunsale': 4,
        'age18longgunsale': 3,
        'age21longgunsaled': 4,
        'age21longgunsale': 4,
        'loststolen': 3,
        'onepermonth': 2,
    },
    'high_risk_gun': {
        'felony': 4,
        'violent': 3,
        'violenth': 3,
        'violentpartial': 2,
        'invcommitment': 4,
        'invoutpatient': 3,
        'danger': 3,
        'drugmisdemeanor': 2,
        'alctreatment': 2,
        'alcoholism': 2,
        'relinquishment': 3,
    },
    'background_checks': {
        'universal': 5,
        'universalh': 4,
        'gunshow': 4,
        'gunshowh': 3,
        'universalpermit': 5,
        'universalpermith': 4,
        'backgroundpurge': 2,
        'threedaylimit': 3,
        'mentalhealth': 4,
        'statechecks': 4,
        'statechecksh': 3,
    },
    'possession_regulations': {
        'age21handgunpossess': 5,
        'age18longgunpossess': 4,
        'age21longgunpossess': 5,
        'gvro': 5,
        'gvrolawenforcement': 4,
        'college': 4,
        'collegeconcealed': 3,
        'elementary': 4,
        'opencarryh': 3,
        'opencarryl': 3,
        'opencarrypermith': 4,
        'opencarrypermitl': 4,
    },
    'assault_weapons': {
        'assault': 5,
        'onefeature': 4,
        'assaultlist': 4,
        'assaultregister': 3,
        'assaulttransfer': 3,
        'magazine': 4,
        'tenroundlimit': 3,
        'magazinepreowned': 3,
    },
    'child_access': {
        'lockd': 4,
        'lockp': 3,
        'lockstandards': 3,
        'locked': 4,
        'capliability': 5,
        'capaccess': 4,
        'capuses': 4,
        'capunloaded': 3,
        'cap18': 4,
        'cap16': 4,
        'cap14': 5,
    },
    'gun_trafficking': {
        'traffickingbackground': 4,
        'traffickingprohibited': 5,
        'traffickingprohibitedh': 4,
        'strawpurchase': 5,
        'strawpurchaseh': 4,
        'microstamp': 4,
        'personalized': 3,
    },
    'domestic_violence': {
        'mcdv': 5,
        'mcdvdating': 5,
        'mcdvsurrender': 4,
        'mcdvsurrendernoconditions': 5,
        'mcdvsurrenderdating': 4,
        'mcdvremovalallowed': 3,
        'mcdvremovalrequired': 4,
        'incidentremoval': 3,
        'incidentall': 4,
        'dvro': 5,
        'dvrodating': 4,
        'exparte': 5,
        'expartedating': 4,
        'dvrosurrender': 4,
        'dvrosurrendernoconditions': 5,
        'dvrosurrenderdating': 4,
        'expartesurrender': 4,
        'expartesurrendernoconditions': 5,
        'expartesurrenderdating': 4,
        'dvroremoval': 4,
        'stalking': 4,
    },
    'concealed_carry' : {
    'permitconcealed': 3,
    'mayissue': 2,
    'showing': 2,
    'ccrevoke': 2,
    'ccbackground': 3,
    'ccbackgroundnics': 3,
    'ccrenewbackground': 2,
    }
}

In [138]:
# Applying the adjusted scoring system
for category, indicator_scores in category_scores.items():
    legislation_db[f'{category}_score'] = legislation_db.apply(
        lambda row: max(row[indicator] * score for indicator, score in indicator_scores.items()),
        axis=1
    )


In [139]:
cols = ['state', 'year', 'lawtotal'] + [f'{category}_score' for category in category_scores.keys()]
legislation_db = legislation_db[cols]
legislation_db.sample(5)

,state,year,lawtotal,dealer_regulations_score,buyers_regulations_score,high_risk_gun_score,background_checks_score,possession_regulations_score,assault_weapons_score,child_access_score,gun_trafficking_score,domestic_violence_score,concealed_carry_score
777,Montana,2018,4,0,0,0,0,4,0,0,0,3,3
1317,Utah,2018,15,0,0,4,0,4,0,0,5,5,3
1107,Oregon,2018,38,5,4,4,5,5,0,0,0,5,3
387,Illinois,2018,65,5,5,4,5,4,0,5,5,5,3
1227,South Dakota,2018,6,0,0,2,0,0,0,0,0,5,3


In [140]:
score_columns = [f'{category}_score' for category in category_scores.keys()]
legislation_db['score_sum'] = legislation_db[score_columns].sum(axis=1)
legislation_db.sample(5)

,state,year,lawtotal,dealer_regulations_score,buyers_regulations_score,high_risk_gun_score,background_checks_score,possession_regulations_score,assault_weapons_score,child_access_score,gun_trafficking_score,domestic_violence_score,concealed_carry_score,score_sum
657,Michigan,2018,21,4,4,4,4,4,0,4,0,0,3,27
897,New Jersey,2018,78,5,5,4,5,5,5,5,3,5,3,45
267,Florida,2018,30,3,4,4,3,4,0,5,5,5,3,36
957,New York,2018,74,4,4,4,5,5,5,4,5,5,3,44
1377,Virginia,2018,13,0,0,4,0,4,0,0,5,5,3,21


In [141]:
category_weights = {
    'dealer_regulations': 0.2,
    'buyers_regulations': 0.15,
    'high_risk_gun': 0.1,
    'background_checks': 0.1,
    'possession_regulations': 0.1,
    'assault_weapons': 0.1,
    'child_access': 0.05,
    'gun_trafficking': 0.05,
    'domestic_violence': 0.1,
    'concealed_carry': 0.15,
}

# Calculate the weighted sum for each row
legislation_db['score_legis'] = 0
for category, weight in category_weights.items():
    legislation_db['score_legis'] += legislation_db[f'{category}_score'] * weight

In [142]:
legislation_db = legislation_db[['state', 'year', 'score_legis']]
legislation_db.sample(5)

,state,year,score_legis
477,Kansas,2018,0.90
687,Minnesota,2018,3.70
357,Idaho,2018,0.40
1077,Oklahoma,2018,2.00
117,Arkansas,2018,1.25


## Récupération du nombre d'armes vendues chaque année par Etat

Pas de données publiques, le proxy le plus fréquent est le nombre de background checks : https://github.com/BuzzFeedNews/nics-firearm-background-checks/tree/master




In [143]:
bg_checks_db = pd.read_csv('data/bg_checks.csv')
bg_checks_db = bg_checks_db[['month', 'state', 'totals']]
bg_checks_db.sample(5)

,month,state,totals
2648,2019-09,District of Columbia,610
11301,2006-08,Mississippi,13709
15490,2000-04,North Carolina,18841
686,2022-09,Mississippi,21612
10242,2008-03,Hawaii,689


In [144]:
bg_checks_db['month'] = pd.to_datetime(bg_checks_db['month'] + '-01')

# Create a new column for the year
bg_checks_db['year'] = bg_checks_db['month'].dt.year

# Create a pivot table
bg_checks_db = bg_checks_db.pivot_table(values='totals', index='state', columns='year', aggfunc='sum', fill_value=0)

# Display the result
bg_checks_db = bg_checks_db.loc[:, 2013:2018]
bg_checks_db.reset_index(inplace=True)

In [145]:
bg_checks_db.columns = ['state', 'bchecks_2013', 'bchecks_2014', 'bchecks_2015', 'bchecks_2016', 'bchecks_2017', 'bchecks_2018']

In [146]:
bg_checks_db.sample(5)

,state,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018
2,Arizona,363036,310672,331442,416279,384930,377838
52,West Virginia,248952,221847,249014,242350,222768,241678
25,Minnesota,525774,481122,519244,651599,683544,604078
13,Idaho,147494,131742,136946,156343,182099,207320
14,Illinois,1280613,1344096,1247398,1924070,1601087,2831447


## Récupération de la part de vote républicain / démocrate

Pour cela nous disponsons d'une base recensant les votes dans chaque comté aux présidentielles de 2016: https://github.com/tonmcg/US_County_Level_Election_Results_08-20/tree/master. Il sera facile ensuite de merge les résultats car cette base renseigne pour chaque ligne le fips (identifiant des comtés.)

In [147]:
vote_db = pd.read_csv('https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-20/master/2016_US_County_Level_Presidential_Results.csv')
vote_db= vote_db[['per_dem', 'per_gop', 'county_name', 'combined_fips']]
vote_db['combined_fips'] = vote_db['combined_fips'].astype(str)
vote_db['combined_fips'] = vote_db['combined_fips'].apply(lambda x: x.zfill(5))
vote_db.sample(5)

,per_dem,per_gop,county_name,combined_fips
736,0.188632,0.771813,Johnson County,17087
918,0.112199,0.848645,Gove County,20063
1125,0.423143,0.558324,Claiborne Parish,22027
877,0.301027,0.652539,Vermillion County,18165
2138,0.211149,0.723421,Canadian County,40017


## Fusion des bases et export

In [148]:
final_db = pd.merge(left=counties_db, right=legislation_db, how='left', left_on='Etat', right_on='state').drop(columns=['state', 'year'])
final_db = pd.merge(left=final_db, right=bg_checks_db, how='left', left_on='Etat', right_on='state').drop(columns=['state'])
final_db = pd.merge(left=final_db, right=vote_db, how='left', left_on='fips', right_on='combined_fips').drop(columns=['combined_fips', 'county_name'])

final_db.sample(5)

,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,unemp_rate_year_2013,unemp_rate_year_2014,...,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop
586,Jerome,Idaho,ID,27294,27920,Jerome,"POLYGON ((-114.61740 42.85089, -114.59518 42.8...",16053,5.2,3.2,...,14.9,0.40,147494,131742,136946,156343,182099,207320,0.198463,0.688760
1973,Pender,North Carolina,NC,27315,29325,Pender,"POLYGON ((-78.27223 34.53838, -78.26719 34.541...",37141,7.2,5.9,...,26.4,3.60,574622,1182349,531876,601445,537813,529916,0.335624,0.639665
2094,Huron,Ohio,OH,27317,29448,Huron,"POLYGON ((-82.84148 41.29002, -82.83583 41.289...",39077,9.7,8.1,...,13.6,2.40,680930,596389,748502,875724,753072,717475,0.290975,0.654693
473,Jenkins,Georgia,GA,27292,27809,Jenkins,"POLYGON ((-82.14794 32.81976, -82.12374 32.883...",13165,10.7,8.8,...,9.3,1.25,527885,484580,566946,612985,541655,549532,0.365194,0.623599
1010,Bourbon,Kentucky,KY,152,28349,Bourbon,"POLYGON ((-84.44266 38.28324, -84.44114 38.283...",21017,6.0,4.1,...,18.9,1.25,1578331,2492184,3218371,3676847,4641480,4912441,0.317484,0.633489


In [149]:
final_db.to_csv('data/final_db.csv')